In [4]:
from config import CURRENT_PERIOD
from sp_soccer_lib import create_team_df_dict, championship_teams, no_draw_frequencies
from sp_soccer_lib.handout_helpers import style, get_country_header, make_link
from sp_soccer_lib.championships import team_stats, load_country
from sp_soccer_lib.simulation import cash_flow_meta
import math

In [5]:
import numpy as np
def cash_flow_meta(cash_flow:list):
    cumulative = np.cumsum(cash_flow)
    print(cumulative)
    min_cum = np.min(cumulative)
    max_cum = np.max(cumulative)
    print(min_cum, max_cum)
    return min_cum, max_cum

In [6]:
cash_flow_meta([-2, -4, -6, 28.5, -2, -4, -6, 21.0, -2, -4, -6, 20.4, -2, 6.4])

[ -2.   -6.  -12.   16.5  14.5  10.5   4.5  25.5  23.5  19.5  13.5  33.9
  31.9  38.3]
-12.0 38.3


(-12.0, 38.3)

In [2]:
def simulate_team(
    team: str,
    country_team_df,
    period: str,
    bet_progr: list,
    threshold: int,
    bet_span: int,
    fixed_odds: int = 3,
    verbose: bool = False,
    abandon_on_first_bucket = False
):
    team_matches_df = country_team_df[team]
    team_matches_df = team_matches_df[team_matches_df['period'] == period]
    if verbose:
        print(team_matches_df.head())
        print(f'Matches in this period: {team_matches_df.shape[0]}')

    team_bet = 0
    team_wins = 0
    cash_flow = []

    next_bet = 0
    abandon = False
    for _, match in team_matches_df.iterrows():
        if abandon_on_first_bucket and abandon:
            print("Discarding Team Because Abandon Option is Enabled")
            break
        
        odds = fixed_odds if math.isnan(match["B365D"]) else match["B365D"]
        
        if verbose:
            # print(match)
            print('--------------------------------')
            print(f'Match is {match["HomeTeam"]}-{match["AwayTeam"]}. Draw gives {odds}. Result for {team} is {match["result"]}')
        
        if next_bet:
            team_bet = team_bet + bet_progr[next_bet-1]
            if verbose:
                print(f'!!! This is bet No {next_bet}. Betting {bet_progr[next_bet-1]} Eur.  !!!')
            cash_flow.append(-1 * bet_progr[next_bet-1])
        else:
            if verbose:
                print('!!! No bet this time !!!')
            # print('!!! No bet this time !!!')
            pass
        if match['FTR'] == 'D':
            if next_bet:
                winnings = bet_progr[next_bet - 1] * odds
                if verbose:
                    print(f'Colourful Ballons!!! We retrieve {winnings}')
                team_wins = team_wins + winnings
                cash_flow.append(winnings)
            next_bet = 0
        else:
            if match['count_no_draw'] >= threshold and match['count_no_draw'] < threshold + bet_span:
                next_bet = match['count_no_draw'] - threshold + 1
            elif match['count_no_draw'] >= threshold + bet_span:
                abandon = True
            else:
                next_bet = 0
                # abandon = True

                # print('!!!!!BET!!!!!!')
        # if row['count_no_draw'] == threshold:
        #     next_bet = 1

    # print(f'{team} bet for {period}: {team_bet}')
    # print(f'{team} wins for {period}: {team_wins}')
    return team_bet, team_wins, cash_flow


In [7]:
# bet_progr = [2,4,8, 16, 32, 64, 128, 256, 512, 1024]
bet_progr = [2,4,6, 9, 13, 20, 30, 45, 68, 103]
# bet_progr = [2,4,8, 16, 32, 64, 128, 256, 512, 1024]
# bet_progr = [2,4,8, 16, 32, 64, 128, 256, 512, 1024]

In [ ]:
countries = ['greece', 'italy']
periods = ['2021', '2122']
threshold_options = [3, 4, 5, 6]
bet_span_options = [3, 4, 5, 6, 7, 8, 9, 10]

test = True

if test:
    countries = ['greece']
    periods = ['2021']
    threshold_options = [3]
    bet_span_options = [3]

abandon = True
verbose = True


In [10]:
from sp_soccer_lib.championships import team_stats, load_country
from sp_soccer_lib import create_team_df_dict

def stats_dataframe(country):
    df = load_country(country)
    team_df = create_team_df_dict(df)
    return team_stats(team_df), team_df

In [13]:
total_bet = 0
total_wins = 0
team_result = []
period_result = []
for country in countries:
    stats_df, team_df = stats_dataframe(country)
    for threshold in threshold_options:
        for bet_span in bet_span_options:
            for period in ['2021', '2122']:
                for _, team_row in stats_df.iterrows():
                    print(team_row.name)
                    team = team_row.name
                    team_bet, team_wins, cash_flow = simulate_team(str(team), team_df, period, bet_progr, threshold, bet_span, verbose=verbose, abandon_on_first_bucket=abandon)
                    print(f'Team bet: {team_bet}')
                    print(f'Team wins: {team_wins}')
                    print(f'Cash Flow: {cash_flow}')
                    total_bet = total_bet + team_bet
                    total_wins = total_wins + team_wins
                    
                    # TODO: Here we need to calculate min and max cumulative cash flow
                    
                    team_result.append({'country': country, 'period': period, 'threshold': threshold, 'span': bet_span, 'team': team, 'bet': team_bet, 'wins': team_wins})

                print(f'Total bet: {total_bet}')
                print(f'Total wins: {total_wins}')
                # print(f'Cash Flow: {cash_flow}')
                period_result.append({'country': country, 'period': period, 'threshold': threshold, 'span': bet_span, 'total_bet': total_bet, 'total_wins': total_wins})
                total_bet = 0
                total_wins = 0
print(team_result)
print(period_result)

2023-11-05 22:32:28.825 | INFO     | sp_soccer_lib:period_stats:97 - Made Aris 2122 Adjustment


Olympiakos
              HomeTeam          AwayTeam FTR  FTHG  FTAG  B365D period  \
Date                                                                     
2020-09-18  Olympiakos  Asteras Tripolis   H   3.0   0.0   6.00   2021   
2020-09-26  Olympiakos       Panetolikos   H   2.0   0.0   5.75   2021   
2020-10-04    Giannina        Olympiakos   D   1.0   1.0   4.75   2021   
2020-10-17  Olympiakos         Atromitos   H   4.0   0.0   4.75   2021   
2020-10-31  Olympiakos           Apollon   H   2.0   0.0   8.50   2021   

            count_draw  count_no_draw result        name   GF   GA  
Date                                                                
2020-09-18           0              1      W  Olympiakos  3.0  0.0  
2020-09-26           0              2      W  Olympiakos  2.0  0.0  
2020-10-04           1              0      D  Olympiakos  1.0  1.0  
2020-10-17           0              1      W  Olympiakos  4.0  0.0  
2020-10-31           0              2      W  Olympiakos

In [4]:
import pandas as pd
pd.DataFrame.from_dict(team_result).to_excel('team.xlsx')
pd.DataFrame.from_dict(period_result).to_excel('period.xlsx')

In [4]:
11 % 3


2